# HW2A: Alignment with IBM Model 1


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import math
import matplotlib.pyplot as plt # graphs and figures
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
from collections import Counter
from itertools import product
import tqdm.notebook
import pickle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Data

We'll start out by using a toy dataset. Please see [these slides](https://cal-cs288.github.io/sp20/slides/cs288_sp20_05_statistical_translation_4up.pdf) for a more complete coverage of IBM Model 1, and feel free to check out Philipp Koehn's book _Statistical Machine Translation_. 

In [2]:
aligned_data = [
    (["das", "haus"], ["the", "house"]),
    (["das", "buch"], ["the", "book"]),
    (["ein", "buch"], ["a", "book"]),
]

## Alignment Model

Fill in the code for IBM Model 1 below. A correct implementation should achieve perplexity 4096 on the first iteration and perplexity around 70 by the tenth iteration, for the toy dataset above. Note that we'll be grading you only on the generated `self.translation_probabilities`, so the probability and perplexity functions only exist for you to check the correctness of your own implementation. You may wish to comment them out during implementation and check that `self.translation_probabilities` looks reasonable instead.

In [3]:
import numpy as np

class IBMModel1:
    def __init__(self, data, num_iterations=10, epsilon=1.0, compute_perplexity=True):
        self.data = data # aligned corpus as shown above
        self.num_iterations = num_iterations # iterations of expectation-maximization
        self.epsilon = epsilon
        self.compute_perplexity = compute_perplexity
        
        # Preprocess bitext data:
        self.source_words, self.target_words = set(), set()
        for (source, target) in self.data:
            self.source_words.update(source)
            self.target_words.update(target)
        
        # Initialize uniform probabilities:
        self.translation_probs = {(s, t): 1.0/len(self.target_words)
                                  for s,t in product(self.source_words, self.target_words)}
        
        # print(self.source_words)
        # print(self.target_words)
        
    def e_step(self):
        # YOUR SOLUTION HERE
        # - Iterate over paired sentences in the data and compute:
        # - (1) counts, the number of times a source word is translated into a target word,
        #       weighted by alignment probabilities
        # - (2) total, the sum of counts over all possible target words
        # See slide 32 for more information: https://cal-cs288.github.io/sp20/slides/cs288_sp20_05_statistical_translation_4up.pdf
        # BEGIN SOLUTION

        counts = {(s, t): 0 for s, t in product(self.source_words, self.target_words)}
        total = {s: 0 for s in self.source_words}

        for (source, target) in self.data:
            s_total_t = {}
            for t in target:
                s_total_t[t] = 0
                for s in source:
                    s_total_t[t] += self.translation_probs[(s, t)]
            for t in target:
                for s in source:
                    add = self.translation_probs[(s, t)] / s_total_t[t]
                    counts[(s, t)] = counts.get((s, t), 0) + add
                    total[s] = total.get(s, 0) + add

        return counts, total
        # END SOLUTION
        
    def m_step(self, counts, total):
        # YOUR SOLUTION HERE
        # - Update self.translation_probs using counts and total
        # BEGIN SOLUTION
        for t in self.target_words:
            for s in self.source_words:
                self.translation_probs[(s, t)] = counts.get((s, t), 0) / total[s]
        # END SOLUTION
        
    def train(self):
        # Run EM for self.num_iterations:
        for idx in tqdm.tqdm(range(self.num_iterations)):
            if self.compute_perplexity: 
                print("Iteration: {} | Perplexity: {}".format(idx, self.perplexity()))
            counts, total = self.e_step()
            self.m_step(counts, total)
        if self.compute_perplexity:
            print("Iteration: {} | Perplexity: {}".format(self.num_iterations, self.perplexity()))

    def probability(self, source, target):
        # YOUR SOLUTION HERE
        # - Use the normalization trick from lecture to efficiently compute probabilities
        # - We'll use self.epsilon here, which is defined in the initialization
        # BEGIN SOLUTION
        # norm = self.epsilon / (len(source) + 1) ** len(target)
        norm = self.epsilon / len(source) ** len(target)
        prob = 1
        for j in range(1, len(target)):
            sum = 0
            for i in range(len(source)):
                sum += self.translation_probs[(source[i], target[j])]
            prob *= sum
        return prob * norm
        # END SOLUTION
        
    def perplexity(self):
        # YOUR SOLUTION HERE
        # - Iterate over each pair of sentences in the dataset
        # - Call self.probability and compute a sum in log space
        # - Feel free to comment this out while testing your initial model
        # BEGIN SOLUTION
        log_probs = []
        for (source, target) in self.data:
            log_probs.append(-math.log(self.probability(source, target), 2))
        # return 2 ** np.mean(log_probs)
        return 2 ** np.sum(log_probs)
        # END SOLUTION
        
    def get_alignment(self, source, target):
        # YOUR SOLUTION HERE
        # - Find the best word alignment for a source, target pair
        # - Output a list of [(source_idx, target_idx)]
        #   For example: (["ein", "buch"], ["a", "book"])
        #   should have an alignment [(0,0), (1,1)]
        # BEGIN SOLUTION
        alignment = []
        for i, s in enumerate(source):
            probs = []
            for j, t in enumerate(target):
                prob = self.translation_probs[(s, t)]
                probs.append(prob)
            alignment.append((i, np.argmax(probs)))
        return alignment
        # END SOLUTION

ibm = IBMModel1(aligned_data)
# ibm.e_step()
ibm.train()
test_align = ibm.get_alignment(["ein", "buch"], ["a", "book"])
assert(test_align == [(0, 0), (1, 1)]), f"incorrect alignment: {test_align}"
with open("example_alignments.pkl", "wb") as outfile:
    pickle.dump(ibm.translation_probs, outfile, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 10/10 [00:00<00:00, 1273.70it/s]

Iteration: 0 | Perplexity: 512.0
Iteration: 1 | Perplexity: 113.7777777777778
Iteration: 2 | Perplexity: 97.51462480142041
Iteration: 3 | Perplexity: 85.78196870733935
Iteration: 4 | Perplexity: 77.79096097780953
Iteration: 5 | Perplexity: 72.59043544199892
Iteration: 6 | Perplexity: 69.30530471868815
Iteration: 7 | Perplexity: 67.27067983454162
Iteration: 8 | Perplexity: 66.02699803455424
Iteration: 9 | Perplexity: 65.27284310122455
Iteration: 10 | Perplexity: 64.81686703974303


## Visualization and Analysis

Write code to visualize alignments and rerun the IBM model on a (very slightly larger) toy dataset:

In [4]:
def visualize_alignment(alignment, src_texts=None, target_texts=None):
    # YOUR SOLUTION HERE
    # BEGIN ALIGNMENT
    for i in range(len(alignment)):
        s_idx, t_idx = alignment[i]
        # print(s_idx, t_idx)
        print(f"{src_texts[s_idx]} ==> {target_texts[t_idx]}")
    # END ALIGNMENT

aligned_data = [
    (['klein', 'ist', 'das', 'haus'], ['the', 'house', 'is', 'small']),
    (['das', 'haus', 'ist', 'ja', 'groß'], ['the', 'house', 'is', 'big']),
    (['das', 'buch', 'ist', 'ja', 'klein'], ['the', 'book', 'is', 'small']),
    (['das', 'haus'], ['the', 'house']),
    (['das', 'buch'], ['the', 'book']),
    (['ein', 'buch'], ['a', 'book'])
]
ibm = IBMModel1(aligned_data)
ibm.train()
alignment = ibm.get_alignment(['klein', 'ist', 'das', 'haus'], ['the', 'house', 'is', 'small'])
visualize_alignment(alignment, ['klein', 'ist', 'das', 'haus'], ['the', 'house', 'is', 'small'])

100%|██████████| 10/10 [00:00<00:00, 1667.65it/s]

Iteration: 0 | Perplexity: 11073029760800.064
Iteration: 1 | Perplexity: 128651404090.76886
Iteration: 2 | Perplexity: 37310128868.217896
Iteration: 3 | Perplexity: 17088090612.401613
Iteration: 4 | Perplexity: 10288974458.354176
Iteration: 5 | Perplexity: 7332416600.185504
Iteration: 6 | Perplexity: 5807037578.352356
Iteration: 7 | Perplexity: 4923823037.108423
Iteration: 8 | Perplexity: 4370929877.625765
Iteration: 9 | Perplexity: 4005806888.655537
Iteration: 10 | Perplexity: 3755272702.3372397
klein ==> small
ist ==> is
das ==> the
haus ==> house


We'll now run the IBM model on a significantly larger dataset to showcase its failure modes:

In [5]:
!pip install sentencepiece torchtext==0.8.1
import sentencepiece
import torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.8/776.8 MB 1.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.7.1 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but yo

In [6]:
# Load the Multi30K translation dataset:
extensions = [".de", ".en"]
source_field = torchtext.data.Field(tokenize=lambda x: x)
target_field = torchtext.data.Field(tokenize=lambda x: x)
training_data, validation_data, test_data = torchtext.datasets.Multi30k.splits(
    extensions, [source_field, target_field], root=".")

/usr/local/lib/python3.8/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 682kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 243kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 242kB/s]
/usr/local/lib/python3.8/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [7]:
def preprocess(sentence):
    sentence = sentence.translate(str.maketrans('', '', string.punctuation)) # strip punctuation
    return sentence.strip().lower().split()

aligned_data = []
for example in training_data[:1000]:
    source = preprocess(example.src)
    target = preprocess(example.trg)
    aligned_data.append((source, target))

ibm = IBMModel1(aligned_data, compute_perplexity=False)
ibm.train()
with open("multi30k_alignments.pkl", "wb") as outfile:
    pickle.dump(ibm.translation_probs, outfile, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 10/10 [01:45<00:00, 10.54s/it]


In [8]:
# Making sure the model learned something:
examples = [
    ("hund", "dog"),
    ("hund", "cat"),
    ("ein", "a"),
    ("ein", "the"),
    ("frau", "woman"),
    ("frau", "man"),
]


for example in examples:
    print(str(example) + ": " + str(ibm.translation_probs[example]))

('hund', 'dog'): 0.9765105915208369
('hund', 'cat'): 1.0745541756409012e-16
('ein', 'a'): 0.904016367677722
('ein', 'the'): 0.0002091328743299106
('frau', 'woman'): 0.9528437741343648
('frau', 'man'): 2.454275253390026e-07


From this larger dataset: find at least one sentence where the IBM alignment model performs reasonably well, and find another one where it fails catastrophically, and include alignment visualizations for both examples in your report. You may want to consult a [German-English dictionary](https://www.collinsdictionary.com/us/dictionary/english-german) for this part of the problem. Provide a brief explanation for why the alignment model did poorly on the failure case.

In [ ]:
print(aligned_data[3])

(['ein', 'mann', 'in', 'einem', 'blauen', 'hemd', 'steht', 'auf', 'einer', 'leiter', 'und', 'putzt', 'ein', 'fenster'], ['a', 'man', 'in', 'a', 'blue', 'shirt', 'is', 'standing', 'on', 'a', 'ladder', 'cleaning', 'a', 'window'])


In [9]:
de, en = aligned_data[3]
align = ibm.get_alignment(de, en)
visualize_alignment(align, de, en)
print("de:\t", de)
print("en:\t", en)

ein ==> a
mann ==> man
in ==> in
einem ==> a
blauen ==> blue
hemd ==> shirt
steht ==> standing
auf ==> on
einer ==> a
leiter ==> ladder
und ==> a
putzt ==> cleaning
ein ==> a
fenster ==> window
de:	 ['ein', 'mann', 'in', 'einem', 'blauen', 'hemd', 'steht', 'auf', 'einer', 'leiter', 'und', 'putzt', 'ein', 'fenster']
en:	 ['a', 'man', 'in', 'a', 'blue', 'shirt', 'is', 'standing', 'on', 'a', 'ladder', 'cleaning', 'a', 'window']


In [10]:
de, en = aligned_data[9]
align = ibm.get_alignment(de, en)
visualize_alignment(align, de, en)
print("de:\t", de)
print("en:\t", en)

jungen ==> boys
tanzen ==> the
mitten ==> the
in ==> in
der ==> the
nacht ==> night
auf ==> on
pfosten ==> poles
de:	 ['jungen', 'tanzen', 'mitten', 'in', 'der', 'nacht', 'auf', 'pfosten']
en:	 ['boys', 'dancing', 'on', 'poles', 'in', 'the', 'middle', 'of', 'the', 'night']


In [11]:
print(ibm.translation_probs[('tanzen', 'dance')])
print(ibm.translation_probs[('tanzen', 'dancing')])
print(ibm.translation_probs[('tanzen', 'the')])

0.294913456168628
0.02111680882816914
0.038346763508053196


In [12]:
de, en = aligned_data[12]
align = ibm.get_alignment(de, en)
visualize_alignment(align, de, en)
print("de:\t", de)
print("en:\t", en)

ein ==> a
schwarzer ==> black
hund ==> dog
und ==> and
ein ==> a
gefleckter ==> spotted
hund ==> dog
kämpfen ==> and
de:	 ['ein', 'schwarzer', 'hund', 'und', 'ein', 'gefleckter', 'hund', 'kämpfen']
en:	 ['a', 'black', 'dog', 'and', 'a', 'spotted', 'dog', 'are', 'fighting']


In [13]:
de, en = aligned_data[43]
align = ibm.get_alignment(de, en)
visualize_alignment(align, de, en)
print("de:\t", de)
print("en:\t", en)

eine ==> a
schöne ==> new
braut ==> bride
geht ==> walking
auf ==> on
einem ==> a
gehweg ==> sidewalk
mit ==> with
ihrem ==> her
neuen ==> new
ehemann ==> new
de:	 ['eine', 'schöne', 'braut', 'geht', 'auf', 'einem', 'gehweg', 'mit', 'ihrem', 'neuen', 'ehemann']
en:	 ['a', 'beautiful', 'bride', 'walking', 'on', 'a', 'sidewalk', 'with', 'her', 'new', 'husband']


In [14]:
print(ibm.translation_probs[('ehemann', 'husband')])
print(ibm.translation_probs[('ehemann', 'new')])

0.28219773889850386
0.28219773889850386


In [15]:
de, en = aligned_data[34]
align = ibm.get_alignment(de, en)
visualize_alignment(align, de, en)
print("de:\t", de)
print("en:\t", en)

eine ==> a
person ==> person
fährt ==> riding
auf ==> on
einer ==> a
verschneiten ==> snowy
straße ==> road
fahrrad ==> bike
de:	 ['eine', 'person', 'fährt', 'auf', 'einer', 'verschneiten', 'straße', 'fahrrad']
en:	 ['a', 'person', 'riding', 'a', 'bike', 'on', 'a', 'snowy', 'road']
